In [ ]:
import lilac as ll

config = ll.DatasetConfig(
  namespace='local',
  name='glaive',
  source=ll.HuggingFaceSource(dataset_name='glaiveai/glaive-code-assistant'),
)
dataset = ll.create_dataset(config)


In [1]:
ll.start_server()


/Users/dsmilkov/code/lilac/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:     Started server process [70417]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5432 (Press CTRL+C to quit)


In [ ]:
import re
import subprocess
import lilac as ll

code_block_re = re.compile('```(py|python)\n(.*)?\n```', re.MULTILINE | re.DOTALL)


# Format the code blocks of the "answer" column using the `ruff`` formatter.
def format_code(item):
  text = item['answer']
  if not text:
    return None

  new_text = text
  has_edit = False
  for _, code_block in code_block_re.findall(text):
    if not code_block:
      continue
    try:
      # Call the ruff binary to format the current code block.
      formatted_code_block = subprocess.check_output(
        ['ruff', 'format', '-'], input=code_block, encoding='utf-8', stderr=subprocess.DEVNULL
      )
      new_text = new_text.replace(code_block, formatted_code_block)
      has_edit = True
    except subprocess.CalledProcessError:
      continue
  return {'answer': new_text, 'has_edit': has_edit}


ds = ll.get_dataset('local', 'glaive')
ds.map(format_code, output_column='answer_formatted', num_jobs=-1, execution_type='processes')
